## Analyze A/B Test Results

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import statsmodels.api as sm
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [ ]:
df = pd.read_csv('ab_data.csv')
df

b. Use the below cell to find the number of rows in the dataset.

In [ ]:
num_rows = df.shape[0] 
num_rows

c. The number of unique users in the dataset.

In [ ]:
users = df['user_id'].unique()
num_users = users.shape[0]
num_users

d. The proportion of users converted.

In [ ]:
p = df['converted'] == 1
p.mean()

e. The number of times the `new_page` and `treatment` don't line up.

In [ ]:
df[(df['group'] == 'treatment') & (df['landing_page'] == 'new_page')].shape[0]

f. Do any of the rows have missing values?

In [ ]:
bool(num_rows - df.dropna().shape[0])

`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [ ]:
df2 = df[((df['group'] == 'control') & (df['landing_page'] == 'old_page')) | ((df['group'] == 'treatment') & (df['landing_page'] == 'new_page'))]
df2

In [ ]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [ ]:
users = df2['user_id'].unique()
num_users = users.shape[0]
num_users

b. There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
df2[df2['user_id'].duplicated()].user_id

c. What is the row information for the repeat **user_id**? 

In [ ]:
df2[df2['user_id'].duplicated()]

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [ ]:
df2[df2['user_id'].duplicated()].index


In [ ]:
df2.drop([2893], axis=0, inplace=True)
df2

`4.` Use **df2** in the below cells to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
(df2['converted'] == 1).mean()

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
cRate_old = df2[df2['group'] == 'control']['converted'].mean()
cRate_old

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
cRate_new = df2[df2['group'] == 'treatment']['converted'].mean()
cRate_new

d. What is the probability that an individual received the new page?

In [ ]:
(df2['landing_page'] == 'new_page').mean()

e. Consider your results from a. through d. above, and explain below whether you think there is sufficient evidence to say that the new treatment page leads to more conversions.

**Answer: As shown by the observed difference, the control group conversion probability is, although with a smaller fraction, is larger than the conversion rate of the treatment group, and the the probability that an individual received the new page = the probability that an individual received the old page, but we still not sure that the old page is better or not because the number of individuals that recieved old page is not equal to individuals who recieved new one and there is other features may affect the conversion rate (we will study them all in part III), so an AB test should be done to further investigate**

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

Answer:

$H_0: p_{new} - p_{old} \leq 0$


$H_1: p_{new} - p_{old} > 0$

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **convert rate** for $p_{new}$ under the null? 

In [ ]:
df2

In [ ]:
p_new = df2['converted'].mean()
p_new

b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [ ]:
p_old = df2['converted'].mean()
p_old

c. What is $n_{new}$?

In [ ]:
n_new = df2[df2['group'] == 'treatment'].shape[0]
n_new

d. What is $n_{old}$?

In [ ]:
n_old = df2[df2['group'] == 'control'].shape[0]
n_old

e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
new_page_converted = np.random.choice([0,1], size=n_new, p=[(1-p_new), (p_new)])
new_page_converted

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
old_page_converted = np.random.choice([0,1], size=n_old, p=[(1-p_old), (p_old)])
old_page_converted

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
new_page_converted.mean() - old_page_converted.mean()

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in a numpy array called **p_diffs**.

In [ ]:
p_diffs = []
for __ in range(10000):
    new_page_converted = np.random.choice([0,1], size=n_new, p=[(1-p_new), p_new]).mean()
    old_page_converted = np.random.choice([0,1], size=n_old, p=[(1-p_old), p_old]).mean()
    p_diffs.append(new_page_converted - old_page_converted)


i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
p_diffs = np.array(p_diffs)

In [ ]:
plt.figure(figsize=(10,10))
plt.hist(p_diffs,label='p_diffs')
plt.legend()
plt.show()

In [ ]:
null_vals = np.random.normal(0, p_diffs.std(), 10000)
obs_diff = cRate_new - cRate_old
obs_diff


In [ ]:
plt.figure(figsize=(10,10))
plt.hist(null_vals, label='Null distribution')
plt.axvline(obs_diff, c='r', label='Observed statistic')
plt.legend()
plt.show()

In [ ]:
(null_vals > obs_diff).mean()

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
(p_diffs > obs_diff).mean()

k. In words, explain what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

Answer:
- P-value
- P-value = 0.905, so we fail to reject the null hypothesis (Old page is better)

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
# number of conversions with the old_page
convert_old = df2[df2['group'] == 'control']['converted'].sum()

# number of conversions with the new_page
convert_new = df2[df2['group'] == 'treatment']['converted'].sum()

# number of individuals who were shown the old_page
n_old = df2[df2['group'] == 'control'].shape[0]

# number of individuals who received new_page
n_new = df2[df2['group'] == 'treatment'].shape[0]


m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

In [ ]:
z_score, p_value = sm.stats.proportions_ztest([convert_new,convert_old], [n_new,n_old], alternative='larger')
print(z_score, p_value)

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**Answer: Yes**

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Answer: Logistic regression**

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df2

In [ ]:
df2['intercept'] = 1
df2 = df2.join(pd.get_dummies(df['group']))
#df2.rename(columns={'treatment':'ab_page'})
df2

c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

In [ ]:
logit_model = sm.Logit(df2['converted'], df2[['intercept','treatment']])
results = logit_model.fit()
results.summary()

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
results.summary()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in the **Part II**?

Answer:
- Hypotheses:<br>

  $H_0: p_{new} - p_{old} = 0$


  $H_1: p_{new} - p_{old} \neq 0$



- As we see, the hypotheses statements here is different from Part II. As in Part II, we asked if the conversion rate of the new page is greater than the old page (one-sided), but here we asked if the conversion rate of the new page is not equal to the old page (two-sided). <br>

- Because the p-value = 0.19 (greater than 0.05), so we fail to reject the null (there is no difference between the conversion rates in both pages), so we can say that there is no significant difference in conversion based on which page a customer receives.

- Untill now, the old page is better but we will consider other things that might influence our result.

<br>



f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

**Answer: Adding more features to our machine learning model will provide more accurate results as they might influence our results, but we must ensure that all features are independent**

<br>


g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
# Read the countries.csv
countries_df = pd.read_csv('./countries.csv')
countries_df


In [ ]:
# Join with the df2 dataframe
df_merged = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')
df_merged

In [ ]:
df_merged['country'].unique()

In [ ]:
# Create the necessary dummy variables
df_merged[['CA', 'UK', 'US']] = pd.get_dummies(df_merged['country'])
df_merged

In [ ]:
# # Fit your model, and summarize the results
df_merged['intercept'] = 1
logit_model_2 = sm.Logit(df_merged['converted'], df_merged[['intercept', 'treatment', 'UK', 'US']])
results = logit_model_2.fit()
results.summary()

**Comment: From the above summary, we find that none of the features is statistically significant as the p-value of all of them is larger than 0.05, so we can say that country doesn't have an impact on conversion.**

<br>


h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
df_merged['UK_treatment'] = df_merged['UK'] * df_merged['treatment']
df_merged['US_treatment'] = df_merged['US'] * df_merged['treatment']


In [ ]:
logit_model_3 = sm.Logit(df_merged['converted'], df_merged[['intercept', 'treatment', 'UK', 'US', 'UK_treatment', 'US_treatment']])
results2 = logit_model_3.fit()
results2.summary()

**Comment: From the above summary, we find that none of the features is statistically significant as the p-value of all of them is larger than 0.05. Finally, so we can say that the interaction between page and country doesn't affect the conversion**

<a id='conclusions'></a>
## Summary and Conclusions

- We first used probability and we conclude that the conversion rate of the old page is slightly higher than the new page but we are not very sure, so we decieded to did an AB test to further investigate

- In our AB test, we set our hypotheses and we found out that the the null hypothesis is true (the old page is better)

- Then we performed many logistic regression models to see if we can get the same result or there will be other features that may affect the conversion rate.

- We performed our first regression model to see if there is a significant difference in conversion based on which page a customer receives and we found out that there is no significant relation between ab page and conversion rate

- We also performed other regression models to see it the country or the interaction between page and country affect the conversion rate, and we found out that they have no effect

- Finally, we conclude that the old page is slightly better than the new page, so we should either keep the old page or provide another new page and study again what we should do.


# Congratulations on completing the project! 

### Gather Submission Materials

Once you are satisfied with the status of your Notebook, you should save it in a format that will make it easy for others to read. You can use the __File -> Download as -> HTML (.html)__ menu to save your notebook as an .html file. If you are working locally and get an error about "No module name", then open a terminal and try installing the missing module using `pip install <module_name>` (don't include the "<" or ">" or any words following a period in the module name).

You will submit both your original Notebook and an HTML or PDF copy of the Notebook for review. There is no need for you to include any data files with your submission. If you made reference to other websites, books, and other resources to help you in solving tasks in the project, make sure that you document them. It is recommended that you either add a "Resources" section in a Markdown cell at the end of the Notebook report, or you can include a `readme.txt` file documenting your sources.

### Submit the Project

When you're ready, click on the "Submit Project" button to go to the project submission page. You can submit your files as a .zip archive or you can link to a GitHub repository containing your project files. If you go with GitHub, note that your submission will be a snapshot of the linked repository at time of submission. It is recommended that you keep each project in a separate repository to avoid any potential confusion: if a reviewer gets multiple folders representing multiple projects, there might be confusion regarding what project is to be evaluated.

It can take us up to a week to grade the project, but in most cases it is much faster. You will get an email once your submission has been reviewed. If you are having any problems submitting your project or wish to check on the status of your submission, please email us at dataanalyst-project@udacity.com. In the meantime, you should feel free to continue on with your learning journey by beginning the next module in the program.

In [ ]:
from subprocess import call 
call(['python', '-m', 'nbconvert', 'project.ipynb'])